# Find Timing for Various Cupy Functions vs. Numpy Functions

## Set up projection image dataset

In [1]:
from tomopyui.backend.io import Projections_Prenormalized
from tomopyui.widgets.imports import PrenormUploader, Import_SSRL62C
import pathlib
import numpy as np

# setting up dummy uploader
dummy_import = Import_SSRL62C()
prenorm_uploader = PrenormUploader(dummy_import)
prenorm_uploader.filedir = pathlib.Path(r"E:\Sam_Welborn\20220620_Welborn\Pristine\all_energies\07660.00eV")
json_file = prenorm_uploader.projections._file_finder(prenorm_uploader.filedir, prenorm_uploader.filetypes_to_look_for)
prenorm_uploader.update_filechooser_from_quicksearch(json_file)
prenorm_uploader.filepath = pathlib.Path(r"E:\Sam_Welborn\20220620_Welborn\Pristine\all_energies\07660.00eV\normalized_projections.hdf5")
prenorm_uploader.filedir = pathlib.Path(r"E:\Sam_Welborn\20220620_Welborn\Pristine\all_energies\07660.00eV")
prenorm_uploader.filename = pathlib.Path(r"normalized_projections.hdf5")

# Import File
prenorm_uploader.projections.import_filedir_projections(prenorm_uploader)
prenorm_uploader.projections._load_hdf_normalized_data_into_memory()
print(f"Data shape = {prenorm_uploader.projections.data.shape}")
print(f"Data type = {prenorm_uploader.projections.data.dtype}")

Data shape = (361, 1024, 1024)
Data type = float32


## Shifting

In [2]:
import numpy as np
import cupy as cp
from cupyx.scipy import ndimage as ndi_cp
from scipy import ndimage as ndi
from skimage import transform as tf
import copy 
import joblib
from time import time

# Set up cupy shifting function
def shift_prj_cp(
    prj,
    sx,
    sy,
    num_batches,
):
    # Why is the error calculated in such a strange way?
    # Will use the standard used in tomopy here, but think of different way to
    # calculate error.

    num_theta = prj.shape[0]
    err = np.zeros((num_theta + 1, 1))

    # split all arrays up into batches.
    err = np.array_split(err, num_batches)
    prj_cpu = np.array_split(prj, num_batches, axis=0)
    sx = np.array_split(sx, num_batches, axis=0)
    sy = np.array_split(sy, num_batches, axis=0)
    for batch in range(len(prj_cpu)):
        _prj_gpu = cp.array(prj_cpu[batch], dtype=cp.float32)
        
        for image in range(_prj_gpu.shape[0]):
            shift_tuple = (sy[batch][image], sx[batch][image])
            _prj_gpu[image] = ndi_cp.shift(_prj_gpu[image], shift_tuple, order=5)

        prj_cpu[batch] = cp.asnumpy(_prj_gpu)

    # concatenate the final list and return
    prj_cpu = np.concatenate(prj_cpu, axis=0)
    err = np.concatenate(err)
    sx = np.concatenate(sx, axis=0)
    sy = np.concatenate(sy, axis=0)
    return prj_cpu

# Shift projections using tomopy's function (skimage), no multiprocessing
def shift_prj_tomopy(prj, sx, sy):
    for m in range(prj.shape[0]):
        tform = tf.SimilarityTransform(translation=(sy[m], sx[m]))
        prj[m] = tf.warp(prj[m], tform, order=5)
        
# Shift projections using tomopy's function (skimage), no multiprocessing
def shift_prj_tomopy_mp(prj, sx, sy, m):
        tform = tf.SimilarityTransform(translation=(sy[m], sx[m]))
        return tf.warp(prj[m], tform, order=5)
        
# Shift projections using ndimage without cupy 
def shift_prj_scipy(prj, sx, sy):
    for m in range(prj.shape[0]):
        shift_tuple = (sy[m], sx[m])
        prj[m] = ndi.shift(prj[m], shift_tuple, order=5)
        
# Shift projections using ndimage without cupy, with multiprocessing
def shift_prj_scipy_mp(prj, sx, sy, m):
    shift_tuple = (sy[m], sx[m])
    return ndi.shift(prj[m], shift_tuple, order=5)
            
# Set up profiling function for tomopy
def benchmark_shift_tomopy(label, prj, sx, sy, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        shifted_prj = shift_prj_tomopy(prj, sx, sy)
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(25)}{t:0.4f}s | avg of {num_runs} runs")
    return shifted_prj

# Set up profiling function for numpy (scipy)
def benchmark_shift_scipy(label, prj, sx, sy, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        shifted_prj = shift_prj_scipy(prj, sx, sy)
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(25)}{t:0.4f}s | avg of {num_runs} runs")
    return shifted_prj

# Set up profiling for tomopy multiprocessing
def benchmark_shift_tomopy_mp(label, prj, sx, sy, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        result = joblib.Parallel(n_jobs=40, backend="threading")(joblib.delayed(shift_prj_tomopy_mp)
                (prj, sx, sy, m) for m in range(prj.shape[0])
          )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(25)}{t:0.4f}s | avg of {num_runs} runs")
    return result

# Set up profiling for scipy multiprocessing
def benchmark_shift_scipy_mp(label, prj, sx, sy, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        result = joblib.Parallel(n_jobs=40, backend="threading")(joblib.delayed(shift_prj_scipy_mp)
                (prj, sx, sy, m) for m in range(prj.shape[0])
          )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(25)}{t:0.4f}s | avg of {num_runs} runs")
    return result

# Set up profiling function for cupy
def benchmark_shift_cp(label, prj, sx, sy, num_batches, num_runs=1, warm_ups=5):
    t = 0.0
    for i in range(warm_ups):
        shifted_prj = shift_prj_cp(prj, sx, sy, num_batches)
        cp.cuda.stream.get_current_stream().synchronize()
    for i in range(num_runs):
        t1 = time()
        shifted_prj = shift_prj_cp(prj, sx, sy, num_batches)
        cp.cuda.stream.get_current_stream().synchronize()
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(25)}{t:0.4f}s | average of {num_runs} runs")
    return shifted_prj

In [ ]:
# Make sure downsampled data not loaded
prenorm_uploader.projections._load_hdf_normalized_data_into_memory()

# Generate random shifts
shift_x = np.random.rand(361) * 30
shift_y = np.random.rand(361) * 30

num_runs = 10
num_batches = 1 # for cutting data up

# Tomopy benchmark
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_shift_tomopy(f"tomopy (skimage.transform.warp): ", prj, shift_x, shift_y, num_runs=num_runs)

# Tomopy benchmark with parallel
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_shift_tomopy_mp(f"tomopy (skimage.transform.warp) with multiprocessing: ", prj, shift_x, shift_y, num_runs=num_runs)

# Scipy benchmark
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_shift_scipy(f"scipy.ndimage.shift: ", prj, shift_x, shift_y, num_runs=num_runs)

# Scipy benchmark with parallel
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_shift_scipy_mp(f"scipy.ndimage.shift with multiprocessing: ", prj, shift_x, shift_y, num_runs=num_runs)

# Scipy-cupy benchmark with parallel
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_shift_cp(f"cupyx.scipy.ndimage.shift: ", prj, shift_x, shift_y, num_batches, num_runs=num_runs)
print("")

## Phase cross correlation

In [3]:
from skimage.registration import phase_cross_correlation as phase_cross_correlation_tomopy
from tomopyui.backend.util.registration._phase_cross_correlation_cupy import phase_cross_correlation as phase_cross_correlation_cp

def pcc_tomopy(prj, sim, upsample_factor):
    for m in range(prj.shape[0]):
        phase_cross_correlation_tomopy(prj[m], sim[m], upsample_factor=upsample_factor, return_error=False)

def pcc_tomopy_mp(prj, sim, upsample_factor, m):
    return phase_cross_correlation_tomopy(prj[m], sim[m], upsample_factor=upsample_factor, return_error=False)

def pcc_cp(prj, sim, upsample_factor, num_batches):
    _prj = np.array_split(prj, num_batches, axis=0)
    _sim = np.array_split(sim, num_batches, axis=0)
    shift_cpu = []
    for batch in range(len(_prj)):
        _prj_gpu = cp.array(_prj[batch], dtype=cp.float32)
        _sim_gpu = cp.array(_sim[batch], dtype=cp.float32)
        shift_gpu = phase_cross_correlation_cp(
            _sim_gpu,
            _prj_gpu,
            upsample_factor=upsample_factor,
            return_error=False,
        )
        shift_cpu.append(cp.asnumpy(shift_gpu))
    shift_cpu = np.concatenate(shift_cpu, axis=1)
    return shift_cpu
    
# Set up profiling function for tomopy's processing
def benchmark_pcc_tomopy(label, prj, sim, upsample_factor, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        pcc_tomopy(prj, sim, upsample_factor)
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")

# Set up profiling for tomopy multiprocessing
def benchmark_pcc_tomopy_mp(label, prj, sim, upsample_factor, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        result = joblib.Parallel(n_jobs=40, backend="threading")(joblib.delayed(pcc_tomopy_mp)
                (prj, sim, upsample_factor, m) for m in range(prj.shape[0])
          )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")
    return result


# Set up profiling function for cupy
def benchmark_pcc_cp(label, prj, sim, upsample_factor, num_batches, num_runs=1, warm_ups=1):
    t = 0.0
    for i in range(warm_ups):
        pcc_cp(prj, sim, upsample_factor, num_batches)
    for i in range(num_runs):
        t1 = time()
        pcc_cp(prj, sim, upsample_factor, num_batches)
        cp.cuda.stream.get_current_stream().synchronize()
        t2 = time()
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | average of {num_runs} runs")

In [5]:
# Generate random shifts
shift_x = np.random.rand(361) * 30
shift_y = np.random.rand(361) * 30
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
sim = joblib.Parallel(n_jobs=40, backend="threading")(joblib.delayed(shift_prj_tomopy_mp)
        (prj, shift_x, shift_y, m) for m in range(prj.shape[0])
  )

In [6]:
import scipy.fft

# Make sure downsampled data not loaded
prenorm_uploader.projections._load_hdf_normalized_data_into_memory()

num_runs = 10
num_batches = 4 # for cutting data up
upsample_factor = 50
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

# # Cupy benchmark
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_pcc_cp(f"Cupy PCC: ", prj, sim, upsample_factor, num_batches, num_runs=num_runs, warm_ups=5)

# Tomopy benchmark with multithreading
scipy.fft.set_global_backend("scipy")
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_pcc_tomopy_mp(f"Tomopy PCC with multiprocessing: ", prj, sim, upsample_factor, num_runs=num_runs)

# Tomopy benchmark
scipy.fft.set_global_backend("scipy")
prj = np.array(copy.deepcopy(prenorm_uploader.projections.data))
benchmark_pcc_tomopy(f"Tomopy PCC without multiprocessing: ", prj, sim, upsample_factor, num_runs=num_runs)

print("")

Cupy PCC:                                         4.6431s | average of 10 runs
Tomopy PCC with multiprocessing:                  7.2724s | avg of 10 runs
Tomopy PCC without multiprocessing:               53.5911s | avg of 10 runs



## Simulate

In [18]:
import tomopyui.tomocupy.recon.algorithm as tomocupy_algorithm
from tomopyui.tomocupy.prep.alignment import simulate_projections as simulate_projections_3D
from tomopy.sim.project import project

In [26]:
def benchmark_simulate_astra(label, rec, num_batches, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        _rec = np.array_split(rec, num_batches, axis=0)
        sim = []
        simulate_projections_3D(
            _rec,
            sim,
            center,
            theta
        )
        sim = np.concatenate(sim, axis=1)
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")
    
def benchmark_simulate_tomopy(label, rec, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        sim = project(rec, theta, pad=False)
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")

In [28]:
# Loading half downsampled data into memory
prenorm_uploader.projections._load_hdf_ds_data_into_memory(pyramid_level=2)
prj = copy.deepcopy(prenorm_uploader.projections.data_ds)
theta = prenorm_uploader.projections.angles_rad
center = prj.shape[1] / 2
num_runs = 10
rec = tomocupy_algorithm.recon_sirt_3D(
    prj,
    theta,
    num_iter=1,
    center=center,
)

# Tomopy multiprocessing projection
benchmark_simulate_tomopy(f"Tomopy Projection with Multithreading: ", rec, num_runs=num_runs)

# Astra 3D projection
num_batches = 5
benchmark_simulate_astra(f"3D Astra Projection: ", rec, num_batches=num_batches, num_runs=num_runs)

Tomopy Projection with Multithreading:            24.1174s | avg of 10 runs
3D Astra Projection:                              0.1626s | avg of 10 runs


## Reconstruct

In [40]:
import tomopy.recon as tomopy_algorithm
from tomopy.recon import wrappers

def benchmark_recon_astra3d(label, rec, recon_iter, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        rec = tomocupy_algorithm.recon_sirt_3D(
            prj,
            theta,
            num_iter=recon_iter,
            center=center,
        )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")
    
def benchmark_recon_astra_plugin(label, rec, recon_iter, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        rec = tomocupy_algorithm.recon_sirt_plugin(
            prj,
            theta,
            num_iter=recon_iter,
            center=center,
        )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")
    
def benchmark_recon_tomopy_astra(label, rec, recon_iter, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        kwargs = {}
        options = {
            "proj_type": "cuda",
            "method": "SIRT_CUDA",
            "num_iter": recon_iter,
        }
        kwargs["options"] = options
        t1 = time()
        rec = tomopy_algorithm(
                    prj,
                    theta,
                    algorithm=wrappers.astra,
                    center=center,
                    ncore=1,
                    **kwargs,
                )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")
    
def benchmark_recon_tomopy(label, rec, recon_iter, num_runs=1):
    t = 0.0
    for i in range(num_runs):
        t1 = time()
        rec = tomopy_algorithm(
                    prj,
                    theta,
                    algorithm="sirt",
                    center=center,
                    ncore=40,
                )
        t2 = time()
        t += t2 - t1
    t = t / num_runs
    print(f"{label.ljust(50)}{t:0.4f}s | avg of {num_runs} runs")

In [42]:
# Loading half downsampled data into memory
prenorm_uploader.projections._load_hdf_ds_data_into_memory(pyramid_level=0)
prj = copy.deepcopy(prenorm_uploader.projections.data_ds)
theta = prenorm_uploader.projections.angles_rad
center = prj.shape[1] / 2
recon_iter = 1
num_runs = 10

benchmark_recon_astra3d(f"SIRT - Astra 3D: ", prj, recon_iter, num_runs=num_runs)

benchmark_recon_astra_plugin(f"SIRT - Astra Plugin: ", prj, recon_iter, num_runs=num_runs)

benchmark_recon_tomopy_astra(f"SIRT - Tomopy Astra Wrapper: ", prj, recon_iter, num_runs=num_runs)

benchmark_recon_tomopy(f"SIRT - Tomopy CPU: ", prj, recon_iter, num_runs=num_runs)

SIRT - Astra 3D:                                  3.1805s | avg of 10 runs
SIRT - Astra Plugin:                              5.5297s | avg of 10 runs
SIRT - Tomopy Astra Wrapper:                      14.9786s | avg of 10 runs
SIRT - Tomopy CPU:                                50.6492s | avg of 10 runs
